## OpenAI Agent SDK Demo

Core concepts:
- Agents: LLMs configured with instructions, tools, guardrails, and handoffs
- Handoffs: A specialized tool call used by the Agents SDK for transferring control between agents
- Guardrails: Configurable safety checks for input and output validation
- Sessions: Automatic conversation history management across agent runs
- Tracing: Built-in tracking of agent runs, allowing you to view, debug and optimize your workflows

The agent loop:  
When you call Runner.run(), we run a loop until we get a final output.

We call the LLM, using the model and settings on the agent, and the message history.
The LLM returns a response, which may include tool calls.
If the response has a final output (see below for more on this), we return it and end the loop.
If the response has a handoff, we set the agent to the new agent and go back to step 1.
We process the tool calls (if any) and append the tool responses messages. Then we go to step 1.
There is a max_turns parameter that you can use to limit the number of times the loop executes.

<p align="center">
    <img src="..\assets\agent-sdk.png" width=700px>
</p>

#### View traces [here](https://platform.openai.com/logs?api=traces) on OpenAI's platform where they enable logging

In [1]:
!pip install openai-agents


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: this example isn't exactly an agent as it has no tool calls but this is just for an example.

In [ ]:
from agents import Agent, Runner
from dotenv import load_dotenv

load_dotenv()

# define the agents
agent = Agent(name="Assistant", 
              instructions="You are a helpful assistant")

# get the runner to loop
result = await Runner.run(agent, 
                          "What is a dog?")
print(result.final_output)

A **dog** is a domesticated mammal and a subspecies of the gray wolf, scientifically known as *Canis lupus familiaris*. Dogs have been companions to humans for thousands of years and are one of the most popular pets worldwide.

**Key facts about dogs:**
- **Species:** Canis lupus familiaris
- **Family:** Canidae (the dog family, which also includes wolves, foxes, and other similar animals)
- **Domestication:** Dogs were domesticated from wolves tens of thousands of years ago.
- **Characteristics:** Most dogs have fur, a keen sense of smell, good hearing, and a relatively strong body. They come in a wide variety of breeds, sizes, shapes, and temperaments.
- **Roles:** Dogs are used for many purposes, including companionship, hunting, herding, protection, service work (like guide dogs for the blind), and police or military work.

Dogs are known for their loyalty, trainability, and ability to bond closely with humans.


### Handoffs example

An agent is an AI model configured with instructions, tools, guardrails, handoffs and more.

We strongly recommend passing instructions, which is the "system prompt" for the agent. In addition, you can pass handoff_description, which is a human-readable description of the agent, used when the agent is used inside tools/handoffs.

Agents are generic on the context type. The context is a (mutable) object you create. It is passed to tool functions, handoffs, guardrails, etc.

#### CAN ADD MAX TURNS!!!!!!!!!!! THIS LIMITS THE AMOUNT OF LOOPS WE DO


In [ ]:
from agents import Agent, Runner

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
    handoff_description="Specialized agent for speaking spanish"
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    handoff_description="Specialized agent for speaking english"
)

# could say this is the principle agent that handles the other 
# task agents as tool calls almost (agent handoff)
triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)

result = await Runner.run(triage_agent, 
                          input="Hola, ¿cómo estás?",
                          max_turns=5   # limit conversation loop to 5 turns
                        )
# CAN ADD MAX TURNS!!!!!!!!!!! THIS LIMITS THE AMOUNT OF LOOPS WE DO

print(result.final_output)

¡Hola! Estoy muy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?


Functions example

In [8]:
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)

result = await Runner.run(agent, input="What's the weather in Tokyo?")
print(result.final_output)
# The weather in Tokyo is sunny.

The weather in Tokyo is currently sunny. If you need more details like temperature or forecast, let me know!


Sessions
--------
The Agents SDK provides built-in session memory to automatically maintain conversation history across multiple agent runs, eliminating the need to manually handle .to_input_list() between turns.

In [5]:
from agents import Agent, Runner, SQLiteSession

# Create agent
agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
)

# Create a session instance
session = SQLiteSession("conversation_123")

# First turn
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session
)
print(result.final_output)  # "San Francisco"

# Second turn - agent automatically remembers previous context
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"

San Francisco.
California.


Session options
No memory (default): No session memory when session parameter is omitted
session: Session = DatabaseSession(...): Use a Session instance to manage conversation history

In [6]:
from agents import Agent, Runner, SQLiteSession

agent = Agent(name="Assistant")

# Different session IDs maintain separate conversation histories
result1 = await Runner.run(
    agent,
    "What's the population?",
    session=session
)

# Custom SQLite database file
session1 = SQLiteSession("user_123", "conversations.db")
result2 = await Runner.run(
    agent,
    "What's the population?",
    session=session1
)

result3 = await Runner.run(
    agent,
    "What is a dog?",
    session=SQLiteSession("user_456", "conversations.db")
)

result4 = await Runner.run(
    agent,
    "Tell me more",
    session=session1
)

print(result1.final_output)
print(result2.final_output)
print(result3.final_output)
print(result4.final_output)

As of the 2020 United States Census, the population of **San Francisco, California** was approximately **815,201** people. More recent estimates (as of 2023) put the population at around **808,000**.
Could you please clarify what location or entity you’re referring to? For example, are you asking about the population of a specific country, city, or region? Let me know so I can give you the most accurate and up-to-date information!
A **dog** is a domesticated mammal with the scientific name **Canis lupus familiaris**. Dogs are closely related to wolves and are believed to have been domesticated by humans more than 15,000 years ago. They are known for their loyalty, companionship, intelligence, and a wide range of breeds with different physical and behavioral traits.

**Key facts about dogs:**
- **Classification:** Mammal, in the family Canidae.
- **Diet:** Omnivorous, but primarily carnivorous.
- **Lifespan:** Varies by breed, typically 10-15 years.
- **Roles:** Companion animal, workin

Custom session implementations
You can implement your own session memory by creating a class that follows the Session protocol (i.e. maybe you have a certain way you do your history truncating):

## TO DO!!!
EXAMPLE IN DOCUMENTATION

### Adding Context

In [12]:
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    """Fetch the age of the user. Call this function to get user's age information."""
    return f"The user {wrapper.context.name} is 47 years old"

# Initialize context
user_info = UserInfo(name="John", uid=123)

agent = Agent[UserInfo](  
    name="Assistant",
    tools=[fetch_user_age],
)

result = await Runner.run(  
    starting_agent=agent,
    input="What is the age of the user?",
    context=user_info,
)

print(result.final_output)  
# The user John is 47 years old.

The user, John, is 47 years old.


## Input Guardrails + agent handoff

Guardrails run in parallel to your agents, enabling you to do checks and validations of user input -> This is why if you look at the trace you can see that the agent ran as well but if the tripwire is triggered we don't display the response.

Creating an AI Agent Tutor with multiple domain specialities + an input guardrail that only answers questions related to homework.

In [40]:
# Define a structured output format
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)
import logging

class HistoryHomeworkOutput(BaseModel):
    is_history_homework: bool
    reasoning: str

# Create a guardrail agent
guardrail_agent = Agent(
    name = "Guardrail Check",
    instructions = "Check if the user is asking about history homework",
    output_type= HistoryHomeworkOutput
)

@input_guardrail
# Define guardrail function
async def history_homework_guardrail(ctx: RunContextWrapper[None], 
                             agent: Agent,
                             input_data: str | list[TResponseInputItem]
                             ) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input_data, context = ctx.context)
    final_output = result.final_output_as(HistoryHomeworkOutput)
    logging.info(final_output)

    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=final_output.is_history_homework
    )


#### Create specialized agents
# Create a science tutor agent
science_tutor_agent = Agent(
    name = "Science Tutor",
    handoff_description="Specialized agent for scientific questions",
    instructions= "You provide assistance with scientific queries. Explain science concepts clearly."
)

# Create a math tutor agent
math_tutor_agent = Agent(
    name = "Math Tutor",
    handoff_description= "Specialist agent for math instrucitons",
    instructions = "You provide help with math problems. Explain your reasoning at each step and include examples."
)

triage_agent = Agent(
    name = "Triage Agent",
    instructions="Handoff to the appropriate agent based on the nature of the request",
    handoffs=[science_tutor_agent, math_tutor_agent],
    input_guardrails=[history_homework_guardrail],
)

In [41]:
# Run the individual agent with a query
# result = await Runner.run(science_tutor_agent, "What element is water?")
# print(result.final_output)
# result = await Runner.run(math_tutor_agent, "What is the derivative of x^2 - 1")
# print(result.final_output)

# run triage agent
# result = await Runner.run(triage_agent, "What is derivative of x^2 - 1")
# print(result.final_output)

result = await Runner.run(guardrail_agent, "What element is water?")
print(result.final_output)

is_history_homework=False reasoning='The user is asking about the chemical nature of water, which is a science (specifically chemistry) question, not about history homework.'


In [42]:
result = await Runner.run(triage_agent, "What element is water?")
print(result.final_output)

Water is **not** an element. Water is a **compound** made up of two elements: **hydrogen** and **oxygen**. Its chemical formula is **H₂O**, which means each molecule of water contains two hydrogen atoms and one oxygen atom bonded together.

- **Element:** A pure substance made of only one kind of atom (e.g., hydrogen, oxygen, gold).
- **Compound:** A substance formed when two or more elements are chemically joined (e.g., water, carbon dioxide).

**Summary:**  
**Water (H₂O) is a compound, not an element.**


In [43]:
# Test with history question (will trigger guardrail)
try:
    result = await Runner.run(triage_agent, "who was the first president of canada?")
    print(result.final_output)
except InputGuardrailTripwireTriggered:
    print("Blocked by input guardrail.")        # only guardrail trips land here
except Exception as e:
    print(f"Error triggered: {type(e).__name__}") # everything else (timeouts, bugs, etc.)

Blocked by input guardrail.
